# Extract Emojis from Datasets

In [62]:
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()
import pickle as pk
import numpy as np
from pathlib import Path
base_dir = os.getenv('BASEDIR')

from tqdm import tqdm
tqdm.pandas()

In [63]:
dataset='qanda'

In [64]:
import emoji

def extract_emojis(s):
    return([o['emoji'] for o in emoji.emoji_list(s)])

In [65]:
dataset_name='qanda'
data = pd.read_csv(os.path.join(base_dir,'data','01_raw_data','qanda','qanda_emoji.csv'), dtype=str, names=['tid', 'rid' ,'qid', 'uid','name', 'bio', 'date', 'text_extended', 'text', 'hashtags', 'hashtags_extended', 'mentions', 'mentions_extended', 'urls_extended', 'urls'], index_col=False)

data['text'] = data['text'].fillna('')
data['bio'] = data['bio'].fillna('')
data['name'] = data['name'].fillna('')

In [66]:
data_per_user = data.groupby('uid').progress_apply(lambda d: pd.DataFrame({'name' : [d['name'].values[0]], 'bio' : [d['bio'].values[0]], 'text' : [' '.join(d['text'])] }))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 103074/103074 [00:48<00:00, 2129.81it/s]


In [67]:
emojis = data_per_user.progress_apply(lambda row: extract_emojis(row['name'] + row['bio'] + row['text']), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 103074/103074 [00:34<00:00, 3021.44it/s]


In [68]:
flags = ['🇦🇺','🇺🇸','🇺🇳','🏳️‍🌈','🏳️‍⚧️','🌈', '💣', '🔪','☕']
for flag in flags:
    data_per_user[flag] = emojis.progress_apply(lambda l: len([x for x in l if x == flag]))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 103074/103074 [00:00<00:00, 887885.11it/s]


In [69]:
data_per_user = data_per_user.reset_index(drop=False)

In [70]:
data_per_user[['uid'] + flags]

,uid,🇦🇺,🇺🇸,🇺🇳,🏳️‍🌈,🏳️‍⚧️,🌈,💣,🔪,☕
0,1000001245946490880,0,0,0,0,0,0,0,0,0
1,1000022700092411904,0,0,0,0,0,0,0,0,0
2,1000111986917572608,0,0,0,0,0,0,0,0,0
3,1000127207048728576,0,0,0,0,0,0,0,0,0
4,1000175125608583168,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
103069,999931831364435968,0,0,0,0,0,0,0,0,0
103070,999937171766235136,0,0,0,0,0,0,0,0,0
103071,999954234119356416,0,0,0,0,0,0,0,0,0
103072,999960633960316929,0,0,0,0,0,0,0,0,0


In [71]:
data_per_user[['uid'] + flags].to_csv(os.path.join(base_dir,'data','01_raw_data',dataset_name,'emoji_count.csv'))